# Custom Data Configurations <a target="_blank" href="https://colab.research.google.com/github/yWorks/yfiles-jupyter-graphs-for-neo4j/blob/main/examples/configurations_example.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install yfiles_jupyter_graphs_for_neo4j --quiet
%pip install neo4j --quiet
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget
from neo4j import GraphDatabase
from datetime import datetime

You can also open this notebook in Google Colab when Google Colab's custom widget manager is enabled:

In [ ]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

## Use the Neo4j driver

In [ ]:
NEO4J_URI      = "neo4j+ssc://demo.neo4jlabs.com" 
NEO4J_USERNAME = "movies"
NEO4J_PASSWORD = "movies"
driver = GraphDatabase.driver(uri = NEO4J_URI, auth = (NEO4J_USERNAME, NEO4J_PASSWORD), database = 'movies')

g = Neo4jGraphWidget(driver)

## Configure data mappings

Configurable values for the node visualization: `text`, `color`, `size`, `styles`, `property`, `coordinate`, `heat` and `type`.

Configurable values for the relationship visualization: `text`, `color`, `thickness_factor`, `styles`, `heat` and `property`.

When adding a configuration, you always need to specify the type of node or edge you want to adjust. (Here 'Person' or 'DIRECTED'),
additionally you can define the new values for the above mentioned properties. The values may either be:
- A lambda the resolves the node or relationship to a specific value.
- A constant value which is directly used.
- A property-key that points to the value that should be used for the configuration.

For nodes, there is an additional `parent_configuration` that defines parent nodes for the node label. This configuration may either be:
- A constant string value that is used as text for the created group node.
- A property-key pointing to a string value that is used as text for the created group node.
- A dict that with a 'text' property as label and additional styling properties (similar to the above mentioned visualization properties).

In [ ]:
g.add_node_configuration('Person', text='name', size=(100,100), styles=lambda node : {'shape': 'rectangle'}, parent_configuration= {'text': 'people', 'color': '#cca9ff'})
g.add_node_configuration('Movie', text= lambda node : {
        'text': 'Title:\n' + node['properties']['title'], 
        'backgroundColor': 'rgba(0,0,0,0.7)', 
        'fontSize': 20, 
        'color': '#FFFFFF', 
        'position': 'north', 
        'fontWeight': 'lighter',
        'maximumWidth': 130, 
        'wrapping': 'word', 
        'textAlignment': 'center'
    }, parent_configuration='movies')  
g.add_relationship_configuration('DIRECTED', text='label', color="blue", styles={'dashStyle': 'dot'})
g.add_relationship_configuration('PRODUCED', text = lambda edge: {'text': 'produced', 'fontWeight': 'bolder'}, color= lambda edge: '#AC94F4')
g.add_relationship_configuration('ACTED_IN', thickness_factor=2, color='#AC94F4') 

## Query the database with Cypher

In [ ]:
g.show_cypher("MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 20")

### Visualize "ACTED_IN" as nested hierarchy

Relationship types may be replaced by a nesting hierarchy. This restructures the graph by removing these relationships and instead representing them as nested parent-child structure. The relationship structure may als be reversed with the `reverse` keyword argument to the `add_parent_relationship_configuration` method.

In [ ]:
g.add_parent_relationship_configuration('ACTED_IN')
g.show_cypher("MATCH (s)-[r]->(t:Movie {title: 'The Matrix'}) RETURN s,r,t LIMIT 20")

## Wildcard configurations

Using `'*'` as the node or relationship type, all nodes or relationships are affected by the given configuration.

Using an explicit type configuration overwrites the wildcard configuration.

In [ ]:
g2 = Neo4jGraphWidget(driver, layout='circular')
g2.add_relationship_configuration('*', color='gray') # all relationship types gray
g2.add_relationship_configuration('PRODUCED', color='red') # highlight the "produced" type
g2.show_cypher("MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 10")